# <font color="green"> Library Imports </font>

In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sqlalchemy import create_engine

import warnings

# <font color="green"> Helper functions </font>

In [47]:
def jupyter_settings():
    
    pd.options.display.max_columns = None   
    


warnings.filterwarnings("ignore")
jupyter_settings()

# 1.0 Data import and basic inspection

## <font color="gray"> 1.1 Data loading and checking </font>

In [48]:
# Data loading

## Importing db credentials

db_credentials = pd.read_csv('/mnt/d/Gustavo/health-insurance/db_credentials.txt', header=None)

host = db_credentials[0][0]

port = db_credentials[1][0]

database = db_credentials[2][0]

user = db_credentials[3][0]

psw = db_credentials[4][0]


In [49]:
# Selecting data from database - SQL query ('insurance' table - pa004 schema)

query = """
    SELECT *
    FROM pa004.insurance

"""

# Connecting to Postgres database

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, psw, host, port, database))


# executing sql query

df_insurance = pd.read_sql_query(query, con=engine)

# closing database connection

engine.dispose()

In [50]:
# Selecting data from database - SQL query ('users' table - pa004 schema)

query = """
    SELECT *
    FROM pa004.users

"""

# Connecting to Postgres database

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, psw, host, port, database))


# executing sql query

df_users = pd.read_sql_query(query, con=engine)

# closing database connection

engine.dispose()

In [55]:
# Selecting data from database - SQL query ('vehicle' table - pa004 schema)

query = """
    SELECT *
    FROM pa004.vehicle

"""

# Connecting to Postgres database

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, psw, host, port, database))


# executing sql query

df_vehicle = pd.read_sql_query(query, con=engine)

# closing database connection

engine.dispose()

In [52]:
# Checking tables

df_insurance.head()

,id,previously_insured,annual_premium,vintage,response
0,1,0,40454.0,217,1
1,2,0,33536.0,183,0
2,3,0,38294.0,27,1
3,4,1,28619.0,203,0
4,5,1,27496.0,39,0


In [53]:
df_users.head()

,id,gender,age,region_code,policy_sales_channel
0,1,Male,44,28.0,26.0
1,2,Male,76,3.0,26.0
2,3,Male,47,28.0,26.0
3,4,Male,21,11.0,152.0
4,5,Female,29,41.0,152.0


In [56]:
df_vehicle.head()

,id,driving_license,vehicle_age,vehicle_damage
0,1,1,> 2 Years,Yes
1,2,1,1-2 Year,No
2,3,1,> 2 Years,Yes
3,4,1,< 1 Year,No
4,5,1,< 1 Year,No


In [ ]:
# Merging tables




In [ ]:
# Data dimensions

In [ ]:
# checking data types

In [ ]:
# Checking data info

In [ ]:
# Checking for NA's

In [ ]:
# Checking for duplicate rows

In [ ]:
# Response variable

In [ ]:
# Response variable - percentage

## <font color="gray"> 1.2 Descriptive statistics </font>

### <i>1.2.1 descriptive statistics</i>

In [ ]:
# Numerical and categorical attributes

In [ ]:
# Descriptive statistics

### <i>1.2.2 general view of attributes</i>

# 2.0 Feature engineering

## <font color="gray"> 2.1 Creating new features </font>

## <font color="gray"> 2.2 Hypothesis to be validated </font>

# 3.0 Data filtering

# 4.0 Exploratory Data Analysis (EDA)

## <font color="gray"> 4.1 Univariate analysis </font>

In [ ]:
# Response variable

### <i>4.1.1 numerical attributes</i>

### <i>4.1.2 categorical attributes</i>

## <font color="gray"> 4.2 Bivariate analysis </font>

### <i>4.2.1 validating hypothesis raised</i>

### <i>4.2.2 statistical hypothesis testing</i>

## <font color="gray"> 4.3 Multivariate analysis </font>

### <i>4.3.1 numerical attributes</i>

### <i>4.3.2 categorical attributes</i>

# 5.0 Data preparation

## <font color="gray"> 5.1 Train/Test split </font>

## <font color="gray"> 5.2 Scaling - numerical attributes </font>

## <font color="gray"> 5.3 Encoding - categorical attributes </font>

# 6.0 Feature selection

## <font color="gray"> 6.1 Boruta algorithm </font>

## <font color="gray"> 6.2 Columns selected by Boruta</font>

# 7.0 Machine learning

## <font color="gray"> 7.1 Baseline model </font>

## <font color="gray"> 7.2 Logistic Regression </font>

## <font color="gray"> 7.3 Random Forest </font>

## <font color="gray"> 7.4 XGBoost </font>

## <font color="gray"> 7.5 KNN </font>

Cross validation

# 8.0 Hyperparameter fine tuning

## <font color="gray"> 8.1 Random search </font>

## <font color="gray"> 8.2 Tuned final model </font>

# 9.0 Error interpretation

## <font color="gray"> 9.1 Main </font>

# 10.0 Deploy

## <font color="gray"> 10.1 Class data_prep.py </font>

## <font color="gray"> 10.2 API_handler.py </font>

## <font color="gray"> 10.3 API tester local </font>

## <font color="gray"> 10.4 API tester remote/cloud </font>